## 相机内参标定 
通过拍摄多张棋盘格照片得到相机内参。

In [4]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

path = 'D:\\labAAA\\printingLight\\process\\camera_set\\chessboard'
list_path = os.listdir(path)
# print(list_path)


w, h = 10, 7
square_size = 15
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
# 准备棋盘格的世界坐标点
objp = np.zeros((w * h, 3), np.float32)
objp[:, :2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)
objp *= square_size


grays = []
pImgs = []
objpoints = []  # 世界坐标点
imgpoints = []  # 图像像素点


# 遍历图像并检测棋盘角点
for file in list_path:
    file_name = os.path.join(path, file)
    img = cv2.imread(file_name)
    scale = 0.5  # 50%缩小
    img_resize = cv2.resize(img, (int(img.shape[1] * scale), int(img.shape[0] * scale)))
    

    gray = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
    # gray_resize = cv2.resize(gray, (int(gray.shape[1] * scale), int(gray.shape[0] * scale)))
    print('image pixel', gray.shape[::-1])
    grays.append(gray)

    ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
    print(f"Chessboard found in {file}: {ret}")
    
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        


        # 可视化检测到的角点（用 matplotlib 替代 OpenCV）
        # img_with_corners = img_resize.copy()
        # cv2.drawChessboardCorners(img_with_corners, (w, h), corners2, ret)
        #使用 matplotlib 显示
        # plt.figure(figsize=(8, 6))
        # plt.imshow(cv2.cvtColor(img_with_corners, cv2.COLOR_BGR2RGB))  # 转换为 RGB 格式
        # plt.scatter(corners2[:, 0, 0], corners2[:, 0, 1], marker='o', c='red', s=10, label="Detected Corners")  # `s`控制点大小
        # plt.title(f"Detected Corners: {file}")
        # plt.rcParams['lines.linewidth'] = 2  # 设置线条宽度
        # plt.axis("off")  # 隐藏坐标轴
        # plt.show()


# 相机标定
print("\nStarting camera calibration...")
# ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
#     objpoints, imgpoints, grays[0].shape[::-1], None, None
# )
ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, grays[0].shape[::-1], None, None,flags=cv2.CALIB_RATIONAL_MODEL | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5
)

# 打印结果
print("相机矩阵 (Camera Matrix):\n", camera_matrix)
print("\n畸变系数 (Distortion Coefficients):\n", distortion_coeffs)

# 计算重投影误差
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], camera_matrix, distortion_coeffs)
    error = cv2.norm(imgpoints[i].squeeze(), imgpoints2.squeeze(), cv2.NORM_L2) / len(imgpoints2)
    mean_error += error

print("\n总平均误差 (Mean Reprojection Error):", mean_error / len(objpoints))


# 保存标定结果
np.savez("calibration_result.npz", camera_matrix=camera_matrix, distortion_coeffs=distortion_coeffs, rvecs=rvecs, tvecs=tvecs)

# for file in list_path:
#     file_name = os.path.join(path, file)
#     img = cv2.imread(file_name)
#     scale = 0.5  # 50%缩小
#     img_resize = cv2.resize(img, (int(img.shape[1] * scale), int(img.shape[0] * scale)))
#     undistorted_img = cv2.undistort(img_resize, camera_matrix, distortion_coeffs)
#     plt.figure(figsize=(8, 6))
#     plt.imshow(cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2RGB))  # 转换为 RGB 格式
#     plt.title(f"Detected Corners: {file}")
#     plt.axis("off")  # 隐藏坐标轴
#     plt.show()


image pixel (2736, 1824)
Chessboard found in IMG_6519.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6520.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6521.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6523.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6525.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6526.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6528.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6529.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6530.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6531.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6532.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6534.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6535.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6541.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6542.JPG: True
image pixel (2736, 1824)
Chessboard foun

## 相机外参计算
在实验前拍摄一张完整的棋盘格pattern，检测格点的像素坐标，并与其在样品坐标系的坐标比对，算出相机外参。计算出的矩阵似乎是从相机位置移动到原点的变换，而非相机在坐标系中的位置，所以正负是相反的。
（目前误差有点大）

### 1 读取棋盘格角点

In [27]:
import cv2
import numpy as np

# 读取图像
image_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\camera_calib'
# image_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\refine_camera_calib'
list_image_path = os.listdir(image_path)
scale_factor = 2  # 你的缩放比例
new_camera_matrix = camera_matrix.copy()
new_camera_matrix[0, 0] *= scale_factor  # fx
new_camera_matrix[1, 1] *= scale_factor  # fy
new_camera_matrix[0, 2] *= scale_factor  # cx
new_camera_matrix[1, 2] *= scale_factor  # cy
print(new_camera_matrix)

board_corners = []

for file in list_image_path:
    file_name = os.path.join(image_path, file)
    img = cv2.imread(file_name)
    w, h = 4,4
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
    if ret:
        # 通过角点亚像素精确化，提高角点位置精度
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100,
                    0.0001)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                    criteria)
        board_corners.append(corners2)
        # 可视化角点
        img_with_corners = cv2.drawChessboardCorners(img, (w, h), corners2,
                                                     ret)
        cv2.imwrite(
            f'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\corners\\{file}.png',
            img_with_corners)
    else:
        print("Chessboard corners not found.")

[[1.00188686e+04 0.00000000e+00 2.76277276e+03]
 [0.00000000e+00 1.00183920e+04 1.69481628e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


2， 计算角点在样品坐标系下的坐标，并用solvepnp算外参

In [38]:
from scipy.spatial.transform import Rotation as R
import numpy as np



#从XYZABC输出变换矩阵
def kuka_to_transformation(pose):
    """将 KUKA 的点位信息转换为 4x4 变换矩阵"""
    # 平移向量
    t = np.array([pose[0],pose[1],pose[2]])

    # 计算旋转矩阵（ZYX顺序）
    rotation = R.from_euler('ZYX', [pose[3],pose[4],pose[5]], degrees=True)  # KUKA的ABC通常以度为单位
    R_matrix = rotation.as_matrix()
    # print(R_matrix)

    # 构造4x4变换矩阵
    T = np.eye(4)
    T[:3, :3] = R_matrix
    T[:3, 3] = t
    # print('T', T)
    return T

#从变换矩阵求旋转之后的棋盘格点在样品坐标系里的坐标
def transform_points(origin_points, T_1, T_2):

    # 将3D点转换为齐次坐标
    origin_points_h = np.hstack(
        [origin_points, np.ones((origin_points.shape[0], 1))])
    print(origin_points_h)
    # 使用矩阵乘法进行坐标变换
    #去掉齐次坐标中的最后一列，转换回普通的3D坐标
    object_points_base_inv = np.dot(T_2, origin_points_h.T)
    object_points_o1 = np.dot(np.linalg.inv(T_1), object_points_base_inv).T
    return object_points_o1[:, :3]


#从XYZABC输出法兰在base里的变换矩阵
def tool2end(pose):
    T_t2f = kuka_to_transformation([-22.45, 6.745, 641.177, 0, 0, 0])
    T_t2b = kuka_to_transformation(pose)
    return T_t2b @ np.linalg.inv(T_t2f)


In [ ]:
#solvepnp
ends = []
rvec_list = []
tvec_list = []
pose_list = [[1181.42, -2733.69, 1500, 57.72, -7.11, 78.67],
             [1181.42, -2733.69, 1500, 57.85, 5.18, 70.95],
             [1181.42, -2733.69, 1500, 57.45, 6.33, 81.56],
             [1181.42, -2733.69, 1500, 63.45, -7.25, 82.23]]


grid_size = 90
grid_extend = [3,1,-1,-3]
grid_extend_j = [-3, -1, 1, 3]

#格点竖着排
points_col = np.array([(i * grid_size, j * grid_size, 0) for i in grid_extend_j
                   for j in grid_extend])
#格点横着排
points_row = np.array([(i * grid_size, j * grid_size, 0) for j in grid_extend
                       for i in grid_extend])
T_o1 = kuka_to_transformation([-170, -1800, 1500, 58, 0, 90])
translation_vec = np.array([0.304,-0.944,-50.498])
# translation_vec = np.array([30,-15,-50])
print(len(points_row))

for i in range(len(board_corners)):
    if i == 1 or i == 2:  #竖排
        cornerPoints = points_col
    else:
        cornerPoints = points_row
    #由于样品坐标系原点（也就是TCP）不在表面的中心，所以要计算格点在样品坐标系内的坐标
    origin_points = cornerPoints + translation_vec
    # 姿态1的变换矩阵
    T_o2 = kuka_to_transformation(pose_list[i])  # 姿态2的变换矩阵，这里用的是第一个位置

    object_points = transform_points(origin_points, T_o1, T_o2)
    objectPoints = np.array(object_points, dtype=np.float64).reshape(-1, 1, 3)

    imagePoints = np.array(board_corners[i],
                           dtype=np.float32).reshape(-1, 1, 2)
    
    _, rvec, tvec, inliners = cv2.solvePnPRansac(objectPoints,
                                                 imagePoints.squeeze(),
                                                 new_camera_matrix,
                                                 distortion_coeffs,reprojectionError=8.0, confidence=0.99, iterationsCount=150)

    if not _:
        print("PnP 求解失败")

    rvec_list.append(rvec)
    tvec_list.append(tvec)
    rotation_matrix, _ = cv2.Rodrigues(rvec)

    print("Rotation Matrix:\n", rotation_matrix)
    print("Translation Vector:\n", tvec)
    img_points2, _ = cv2.projectPoints(object_points, rvec, tvec,
                                       new_camera_matrix, distortion_coeffs)

    # 如果 image_points 是 n x 1 x 2 的格式，调整为 n x 2 的格式
    image_points = np.array(imagePoints, dtype=np.float32).reshape(-1, 2)

    # 如果 imgpoints2 是 n x 1 x 2 的格式，调整为 n x 2 的格式
    img_points2 = np.array(img_points2, dtype=np.float32).reshape(-1, 2)
    # 计算重投影误差
    error = cv2.norm(image_points.squeeze(), img_points2.squeeze(),
                     cv2.NORM_L2) / len(img_points2)
    print("重投影误差 (Mean Reprojection Error):", error)

16
[[ 270.304  269.056  -50.498    1.   ]
 [  90.304  269.056  -50.498    1.   ]
 [ -89.696  269.056  -50.498    1.   ]
 [-269.696  269.056  -50.498    1.   ]
 [ 270.304   89.056  -50.498    1.   ]
 [  90.304   89.056  -50.498    1.   ]
 [ -89.696   89.056  -50.498    1.   ]
 [-269.696   89.056  -50.498    1.   ]
 [ 270.304  -90.944  -50.498    1.   ]
 [  90.304  -90.944  -50.498    1.   ]
 [ -89.696  -90.944  -50.498    1.   ]
 [-269.696  -90.944  -50.498    1.   ]
 [ 270.304 -270.944  -50.498    1.   ]
 [  90.304 -270.944  -50.498    1.   ]
 [ -89.696 -270.944  -50.498    1.   ]
 [-269.696 -270.944  -50.498    1.   ]]
Rotation Matrix:
 [[-0.99971029 -0.01070409 -0.02155828]
 [ 0.01035215 -0.9998124   0.01637092]
 [-0.02172947  0.016143    0.99963355]]
Translation Vector:
 [[  -4.22281347]
 [   4.9698178 ]
 [1548.18340163]]
重投影误差 (Mean Reprojection Error): 0.15147604171914233
[[-269.696  269.056  -50.498    1.   ]
 [-269.696   89.056  -50.498    1.   ]
 [-269.696  -90.944  -50.498    

### 3 计算机器人路径中的所有变换
暂时没用到

### 测试外参结果的准确度
使用外参标定时的另一张照片

In [35]:
corner = np.array([
    [90, 90, 0],
    [90, -90, 0],
    [-90, -90, 0],
    [-90, 90, 0]
], dtype=np.float32) + translation_vec#由于样品坐标系原点（也就是TCP）不在表面的中心，所以要计算格点在样品坐标系内的坐标
# corner = np.array([ 
#     [90, 90, 0],
#     [90, -90, 0],
#     [-90, -90, 0],
#     [-90, 90, 0]
# ], dtype=np.float32) 
#计算旋转以后的样品角点，这里只取了第一个做实验
T = kuka_to_transformation([-222.92, -1884.805, 1400, 32.616, 12.209, 70.802])#第一个路径点
# T = kuka_to_transformation([-222.92, -1884.805, 1450, 32.616, 12.209, 70.802])#第2路径点
#取标定外参时拍的另一张来实验
# T_test = kuka_to_transformation(1181.42, -2733.69, 1500, 57.72, -7.12, 78.67)#第一个标定点
# T = kuka_to_transformation(1181.42, -2733.69, 1500, 57.85, 5.18, 70.95)#第二个标定点
# T_test = kuka_to_transformation(1181.42, -2733.69, 1500, 63.45, -7.25, 82.23)
# corner_points = transform_points(corner,T_o1,T_test) + np.array([21.1771,-17.55,-6.745])
corner_points = transform_points(corner,T_o1,T)

print('角点坐标',corner_points)
#计算投影在成像平面上的像素点


#绘制四边形
def draw_quadrilateral(image, points, output_path):
    """
    在图像上绘制四边形并保存结果

    参数:
    image: 输入图像
    points: 四个顶点坐标的列表 [(x1,y1), (x2,y2), (x3,y3), (x4,y4)]
    output_path: 输出图像的保存路径
    """
    # 将输入点转换为numpy数组
    pts = np.array(points, dtype=np.int32)

    # 在图像上绘制四边形
    cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=5)

    # 保存结果
    cv2.imwrite(output_path, image)


# 使用示例:
if __name__ == "__main__":
    # 读取图像
    # image = cv2.imread('D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\camera_calib\\IMG_6564.JPG')
    
    

    for i in range(len(rvec_list)):
        image = cv2.imread('D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\images\\IMG_6567.JPG')
        project_corner_points2, _ = cv2.projectPoints(corner_points, rvec_list[i], tvec_list[i], new_camera_matrix, distortion_coeffs)
        print(project_corner_points2)

        points = project_corner_points2.tolist()
        
        # 输出图像的保存路径
        # output_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\camera_calib\\draw_IMG_6564.JPG'
        output_path = f'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\images\\draw_IMG_6567_{i}.JPG'
        
        # 在图像上绘制四边形并保存
        draw_quadrilateral(image, project_corner_points2, output_path)

角点坐标 [[  25.67211633  -53.12600018  -25.52633816]
 [ -32.1800996  -219.27110032   12.53843312]
 [-191.1239361  -181.20500164  -62.87926354]
 [-133.27172017  -15.05990151 -100.94403482]]
[[[2573.30497746 2076.34619558]]

 [[2955.86509437 3135.77763451]]

 [[4044.25729262 2929.0101417 ]]

 [[3670.3723613  1812.24239608]]]
[[[2515.53116845 2056.91108782]]

 [[2899.76402163 3119.35885332]]

 [[3986.10104652 2913.16217614]]

 [[3609.94738164 1792.6361359 ]]]
[[[2528.5752413  2111.94365188]]

 [[2912.9289926  3170.0995195 ]]

 [[3998.1680432  2967.12048894]]

 [[3622.27943535 1851.17222186]]]
[[[2532.99625238 2091.54836187]]

 [[2917.71304761 3150.16760977]]

 [[4002.99630733 2943.86751516]]

 [[3626.568811   1827.67556918]]]


In [9]:
import numpy as np
import itertools
from scipy.spatial.transform import Rotation as R

#欧拉角组合
degree = np.array([-20,-10,0,10,20])
euler_combos = list(itertools.product(degree,repeat = 3))
print(euler_combos)
#旋转矩阵
rotations = [R.from_euler('zyx', angles,degrees=True) for angles in euler_combos]
#网格点
motion_grid_size = 50
motion_grid_extend = [-2,-1,0,1,2]
motion_grid = np.array([(i*motion_grid_size,j*motion_grid_size,0) for i in motion_grid_extend for j in motion_grid_extend])
# print(motion_grid)
#计算相机相对于样品的坐标
points = []
#相机坐标，并构建齐次坐标
# camera = np.array([ -157.26308306, -234.46892435, -1723.93744597])
camera = -tvec
camera_h = np.append(camera,1)

for rotation in rotations:
    for g in motion_grid:
        T = np.eye(4)
        T[:3, :3] = rotation.as_matrix()
        T[:3, 3] = g
        point = np.dot(np.linalg.inv(T),camera_h)
        points.append(point)

# print(points)  

[(-20, -20, -20), (-20, -20, -10), (-20, -20, 0), (-20, -20, 10), (-20, -20, 20), (-20, -10, -20), (-20, -10, -10), (-20, -10, 0), (-20, -10, 10), (-20, -10, 20), (-20, 0, -20), (-20, 0, -10), (-20, 0, 0), (-20, 0, 10), (-20, 0, 20), (-20, 10, -20), (-20, 10, -10), (-20, 10, 0), (-20, 10, 10), (-20, 10, 20), (-20, 20, -20), (-20, 20, -10), (-20, 20, 0), (-20, 20, 10), (-20, 20, 20), (-10, -20, -20), (-10, -20, -10), (-10, -20, 0), (-10, -20, 10), (-10, -20, 20), (-10, -10, -20), (-10, -10, -10), (-10, -10, 0), (-10, -10, 10), (-10, -10, 20), (-10, 0, -20), (-10, 0, -10), (-10, 0, 0), (-10, 0, 10), (-10, 0, 20), (-10, 10, -20), (-10, 10, -10), (-10, 10, 0), (-10, 10, 10), (-10, 10, 20), (-10, 20, -20), (-10, 20, -10), (-10, 20, 0), (-10, 20, 10), (-10, 20, 20), (0, -20, -20), (0, -20, -10), (0, -20, 0), (0, -20, 10), (0, -20, 20), (0, -10, -20), (0, -10, -10), (0, -10, 0), (0, -10, 10), (0, -10, 20), (0, 0, -20), (0, 0, -10), (0, 0, 0), (0, 0, 10), (0, 0, 20), (0, 10, -20), (0, 10, -10)